In [15]:
# import packages needed to run the code
# style as following: import ["package name"] (as [variable name]) ie. we call a function from matplotlib.pyplot as plt.function
#                     from ["package name"] import ["function"]
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import matplotlib.dates as dates
import glob
import pandas as pd
import os
from scipy.interpolate import CubicSpline
import datetime
import csv
from dateutil.parser import parse
import math
import ipywidgets as widgets
from ipywidgets import interactive


In [9]:
# Get data file names from path with a csv format
path = r'.\2022-2023'
filenames = glob.glob(path + "/*.csv")

In [10]:
# Print names of all files in the path with csv format
names = [os.path.splitext(x)[0] for x in [os.path.basename(x) for x in filenames]]
print(names)
# Print number of files in the path with csv format
print("Number of Sensors= " + str( int(len(names))))

['North_202', 'North_203', 'North_205', 'Outdoor_1', 'South_219', 'South_221', 'South_222', 'South_223', 'South_224', 'South_226']
Number of Sensors= 10


In [11]:
# Print full filename with location (path)
print(filenames)

# Function below tries to find if something is of date-time format (just defining, not running the code)
# Create function that looks at a string (collection of letters and numbers (not treated as a number)) and tries to 
# determine if it is a date. If it is a date, the function will return/output True, otherwise False
# is_date = function name, input= string, fuzzy (fuzzy is automatically set to False unless specified as True)
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
    

['.\\2022-2023\\North_202.csv', '.\\2022-2023\\North_203.csv', '.\\2022-2023\\North_205.csv', '.\\2022-2023\\Outdoor_1.csv', '.\\2022-2023\\South_219.csv', '.\\2022-2023\\South_221.csv', '.\\2022-2023\\South_222.csv', '.\\2022-2023\\South_223.csv', '.\\2022-2023\\South_224.csv', '.\\2022-2023\\South_226.csv']


In [12]:
#combine all files and covert all datetime sequences to float for easier handling of cubic spline.
# define dataframe called dfs as []=empty
dfs = []

# for loop going through each of filenames. The filenames are printed in the block above. For the first loop,
# the variable filename will be set to the first element of filenames (printed above).
# (in this case '.\\2022-2023\\North_202.csv'). Then once we run the code in the loop, we will set the variable
# filename to the second element in filenames (printed above) and continue until we reach the last element in filenames
for filename in filenames:
    print(filename)
    # set variable head_index = 0 (just a number)
    head_index = 0
    # open filename which is of csv format and call the contents of the file as the variable csv_file
    with open (filename,'r') as csv_file:
        # read in the data from the csv file
        reader =csv.reader(csv_file)
        # for loop going through each row in the csv file. We start with the first row and set the variable 'row' as the first row.
        # We run the code in the for loop and continue until the last row (or until break *****)
        for row in reader:
            # if the row is not blank
            if row != []:
                # if the first column in that row is of date-time format we break (meaning stop the for loop *****)
                if is_date(row[0]):
                    #print(head_index)
                    break
                # otherwise we add 1 to head_index
                head_index = head_index + 1
            # if the row is blank add 1 to head_index
            else:
                head_index = head_index + 1

    # Since the [head_index] row (where head_index is a number (the first row with date-time format)) contains the first data point, the head must be the row above this row. Therefore
    # we subtract 1 from head_index so that the [head_index] row is now the header row (= title of columns i.e Date Time, Temperature Humidity (%RH))
    head_index = head_index - 1
    
    # set tail_index = 0 (just a number)
    tail_index = 0
    # open filename which is of csv format and call the contents of the file as the variable csv_file
    with open (filename,'r') as csv_file:
        # read in the data from the csv file
        reader =csv.reader(csv_file)
        # for loop going in the reverse order of the rows. Starts with the variable 'row' equal to the last row of the csv file, and
        # continues with the second to last row until we get to the first row ( or until break *****)
        for row in reversed(list(csv.reader(csv_file))):
            # if row is not empty
            if row != []:
                # if the first column of the row is of date-time format we break (meaning stop the for loop *****)
                if is_date(row[0]):
                    #print(tail_index)
                    break
                # otherwise we add 1 to tail_index
                tail_index = tail_index + 1
            # if the row is blank we add 1 to tail_index
            else:
                tail_index = tail_index + 1

    # store in python the data in csv file as df (data frame). We specify the header as head_index, meaning the [head_index] row contains
    # the column titles, and only the data below this row will be read in. usecols=[i for i in range(3)] makes sure we only read in the first 3 columns.
    df= pd.read_csv(filename, header=head_index, on_bad_lines='skip', infer_datetime_format=True, skip_blank_lines=False, usecols=[i for i in range(3)], low_memory = False)
    # drop the last [tail_index] rows befor reaching the last date-time format row
    df.drop(df.tail(tail_index).index,inplace = True)
    
    #replace the column names with more friendly and shortened descriptors (inplace does not make a new copy of the data
    # , but overwrites the old copy to save memory)
    df.rename(columns={df.columns[0]: 'Date', df.columns[1]: 'Temp', df.columns[2]: 'RH'}, inplace=True)
    # make data in 'Date' column date-time format
    df['Date'] = pd.to_datetime(df['Date'])
    # make data in 'Temp' column as numeric format
    df['Temp']= pd.to_numeric(df["Temp"])
     # make data in 'Temp' column as numeric format
    df['RH']= pd.to_numeric(df["RH"])
    # add df (data from csv file 'filename') to the variable dfs (which was initialized at the top of this block as empty)
    # at the end of the loop dfs will contain the data from all files in 'filenames' (printed in the block above)
    dfs.append(df)
    # go back to the top of the for loop and redo with the next filename in filenames ('.\\2022-2023\\North_203.csv')
    # df = individual file, dfs = collection of files 

.\2022-2023\North_202.csv
.\2022-2023\North_203.csv
.\2022-2023\North_205.csv
.\2022-2023\Outdoor_1.csv
.\2022-2023\South_219.csv
.\2022-2023\South_221.csv
.\2022-2023\South_222.csv
.\2022-2023\South_223.csv
.\2022-2023\South_224.csv
.\2022-2023\South_226.csv


In [26]:


#downsample data for a sensor before plotting

num_samp = str('1D')


%matplotlib inline

#Plot Data
def f(sensor):
    
    #Average RH
    AvRH = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().RH 
    #Standard error RH
    Std_RH = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().RH 
    #Average Temp
    AvT = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Temp #average T for Main Gallery 
    #Standard error Temp
    Std_T = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().Temp #standard error across all sensor and downsamples
    #Average Date
    Avdate = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Date 

    #Calculate standard deviation interval of RH and Temp
    PlusRHSTD=[x+y for x,y in zip(AvRH, Std_RH)]
    MinusRHSTD=[x-y for x,y in zip(AvRH, Std_RH)]
    PlusTSTD=[x+y for x,y in zip(AvT, Std_T)]
    MinusTSTD=[x-y for x,y in zip(AvT, Std_T)]
    
    
    f, ax = plt.subplots(2,1,figsize=(10,9),sharex=True)

    ax[0].axes.get_xaxis().set_visible(False) #set X axis 
    ax[0].axes.get_yaxis().set_visible(True) #set Y axis
    ax[0].scatter(Avdate,AvRH, s = 2, color="None", edgecolor ="blue", alpha =1)
    ax[0].fill_between(Avdate,MinusRHSTD,PlusRHSTD, color = "blue", edgecolor ="None", alpha =0.3 )
    ax[0].set_title(names[sensor], size = 14)
    ax[0].set_ylabel('% RH', size = 14)
    ax[0].axhline(50,linestyle='--', lw=1, color="blue", alpha =0.3)
    ax[0].fill_between(Avdate,45,55, color="blue", edgecolor ="None", alpha =0.05 )
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(True)
    ax[0].set_ylim([0,100])


    ax[1].axes.get_xaxis().set_visible(True) #set X axis 
    ax[1].axes.get_yaxis().set_visible(True) #set Y axis
    ax[1].scatter(Avdate,AvT, s = 2, color="None", edgecolor ="red", alpha =0.8)
    ax[1].fill_between(Avdate,MinusTSTD,PlusTSTD, color="red", edgecolor ="None", alpha =0.3)
    ax[1].axhline(20,linestyle='--', lw=1, color="blue", alpha =0.3)
    ax[1].fill_between(Avdate,18,22, color="blue", edgecolor ="None", alpha =0.05)
    ax[1].set_ylabel('Temp (C)', size = 14)
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_visible(True)
    ax[1].spines['left'].set_visible(True)
    ax[1].set_ylim([10,38])


    f.tight_layout()
    plt.show()

interactive_plot = interactive(f, sensor=(0, len(names)-1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=4, description='sensor', max=9), Output()), _dom_classes=('widget-intera…

In [30]:
def f2(sensor):
    
    #Average RH
    AvRH = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().RH 
    #Standard error RH
    Std_RH = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().RH 
    #Average Temp
    AvT = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Temp #average T for Main Gallery 
    #Standard error Temp
    Std_T = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().Temp #standard error across all sensor and downsamples
    #Average Date
    Avdate = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Date 

    ###########################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    # Change [3] which is outdoor environment to # of sensor you want in the lines below
    #Average RH
    AvRH3 = dfs[3].set_index('Date').resample(num_samp).mean().reset_index().RH 
    #Standard error RH
    Std_RH3 = dfs[3].set_index('Date').resample(num_samp).std().reset_index().RH 
    #Average Temp
    AvT3 = dfs[3].set_index('Date').resample(num_samp).mean().reset_index().Temp #average T for Main Gallery 
    #Standard error Temp
    Std_T3 = dfs[3].set_index('Date').resample(num_samp).std().reset_index().Temp #standard error across all sensor and downsamples
    #Average Date
    Avdate3 = dfs[3].set_index('Date').resample(num_samp).mean().reset_index().Date 
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################

    #Calculate standard deviation interval of RH and Temp
    PlusRHSTD=[x+y for x,y in zip(AvRH, Std_RH)]
    MinusRHSTD=[x-y for x,y in zip(AvRH, Std_RH)]
    PlusTSTD=[x+y for x,y in zip(AvT, Std_T)]
    MinusTSTD=[x-y for x,y in zip(AvT, Std_T)]
    
    ###########################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    #Caculate standard deviation interval of RH and Temp for sensor #3
    PlusRHSTD3=[x+y for x,y in zip(AvRH3, Std_RH3)]
    MinusRHSTD3=[x-y for x,y in zip(AvRH3, Std_RH3)]
    PlusTSTD3=[x+y for x,y in zip(AvT3, Std_T3)]
    MinusTSTD3=[x-y for x,y in zip(AvT3, Std_T3)]
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################
    
    
    f2, ax = plt.subplots(2,1,figsize=(10,9),sharex=True)

    ax[0].axes.get_xaxis().set_visible(False) #set X axis 
    ax[0].axes.get_yaxis().set_visible(True) #set Y axis
    # s=siize of dots, color=none uses edgecolor, edgecolor=boundary of circle, and alpha=transparency of circle
    ax[0].scatter(Avdate,AvRH, s = 2, color="None", edgecolor ="blue", alpha =1)
    ax[0].fill_between(Avdate,MinusRHSTD,PlusRHSTD, color = "blue", edgecolor ="None", alpha =0.3 )
    ############################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    ax[0].scatter(Avdate3,AvRH3, s = 2, color="None", edgecolor ="green", alpha =1)
    ax[0].fill_between(Avdate3,MinusRHSTD3,PlusRHSTD3, color = "green", edgecolor ="None", alpha =0.3 )
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################
    ax[0].set_title(names[sensor], size = 14)
    ax[0].set_ylabel('% RH', size = 14)
    # add horizontal line at y = 50, linestyle is dotted ("--"), lw = lineweight (thickness of line)
    ax[0].axhline(50,linestyle='--', lw=1, color="blue", alpha =0.3)
    # fill graph with blue color between 45 and 55 for ideal RH
    ax[0].fill_between(Avdate,45,55, color="blue", edgecolor ="None", alpha =0.05 )
    # set data boundary lines invisible
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(True)
    ############################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    # set y axis range
    ax[0].set_ylim([0,100])
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################


    ax[1].axes.get_xaxis().set_visible(True) #set X axis 
    ax[1].axes.get_yaxis().set_visible(True) #set Y axis
    ax[1].scatter(Avdate,AvT, s = 2, color="None", edgecolor ="red", alpha =0.8)
    ax[1].fill_between(Avdate,MinusTSTD,PlusTSTD, color="red", edgecolor ="None", alpha =0.3)
    ############################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    ax[1].scatter(Avdate3,AvT3, s = 2, color="None", edgecolor ="green", alpha =0.8)
    ax[1].fill_between(Avdate3,MinusTSTD3,PlusTSTD3, color="green", edgecolor ="None", alpha =0.3)
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################
    ax[1].axhline(20,linestyle='--', lw=1, color="blue", alpha =0.3)
    ax[1].fill_between(Avdate,18,22, color="blue", edgecolor ="None", alpha =0.05)
    ax[1].set_ylabel('Temp (C)', size = 14)
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_visible(True)
    ax[1].spines['left'].set_visible(True)
    ############################################################################################################
    ############################################################################################################
    # new code start
    ############################################################################################################
    ############################################################################################################
    ax[1].set_ylim([10,38])
    ############################################################################################################
    ############################################################################################################
    # new code end
    ############################################################################################################
    ############################################################################################################

    # adjusts spacing between plots (RH and T)
    f2.tight_layout()
    # shows plots = which means print
    plt.show()
 # applying sensors into f2 definition (see def f2 (sensor))   
interactive_plot = interactive(f2, sensor=(0, len(names)-1))
# saves the last sensor as the variable output
output = interactive_plot.children[-1]
interactive_plot


interactive(children=(IntSlider(value=4, description='sensor', max=9), Output()), _dom_classes=('widget-intera…

In [28]:
%matplotlib inline 

num_samp = str('1D')

def g(sensor):
    #Average RH
    AvRH = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().RH 
    #Standard error RH
    Std_RH = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().RH 
    #Average Temp
    AvT = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Temp #average T for Main Gallery 
    #Standard error Temp
    Std_T = dfs[sensor].set_index('Date').resample(num_samp).std().reset_index().Temp #standard error across all sensor and downsamples
    #Average Date
    Avdate = dfs[sensor].set_index('Date').resample(num_samp).mean().reset_index().Date 
    
    counts, xedges, yedges = np.histogram2d(AvT, AvRH, bins=40)
    
    fig, ax = plt.subplots(1,2,figsize=(20,10))
    #sns.despine(left=True, bottom=True, right=True)

    #2D histogram density plot
    ax[0].set_title('Temp-RH ' + names[sensor], size = 24)
    #ax[0].set_facecolor((150/255, 150/255, 150/255)) #set gray background
    ax[0].scatter(AvT,AvRH, s = 30, color="None", edgecolor ="black", alpha =0.8)

    #Color Scatter with sampled pixels
    ax[1].set_title('Density map', size = 24)
    ax[1].imshow(np.rot90(counts))

    plt.show()

interactive_plot2 = interactive(g, sensor=(0, len(names)-1))
output = interactive_plot2.children[-1]
interactive_plot2

interactive(children=(IntSlider(value=4, description='sensor', max=9), Output()), _dom_classes=('widget-intera…

In [17]:
#Find and print longest column
l=[]

for i in range(len(dfs)):
    date = dfs[i][['Date']]
    l.append(len(date)) #find length of vector and append

longest_column = names[l.index(max(l))]

#apply cubic spline interpolation to entire dataset- based on longest column- 
#for even sampling for all galleries or rooms.


for i in range(len(dfs)):
    cs_T = CubicSpline(dfs[i].Date,dfs[i].Temp)
    cs_RH = CubicSpline(dfs[i].Date,dfs[i].RH)
    index = pd.RangeIndex(start=0, stop=max(l))
    dfs[i] = dfs[i].reindex(index).fillna(method='ffill') #reindex dataframe
    dfs[i].Date = dfs[l.index(max(l))].Date
    dfs[i].Temp = cs_T(dfs[l.index(max(l))].Date)
    dfs[i].RH = cs_RH(dfs[l.index(max(l))].Date)

#convert date floats back to date time format   
for i in range(len(dfs)):
    dfs[i][['Date']]=dates.num2date(dfs[i][['Date']])
   

ValueError: `x` must be strictly increasing sequence.